In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Flatten, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, ReLU, Dense, Input
from tensorflow.keras.models import Model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.utils import get_custom_objects

In [6]:
def train_dg(path):

    train = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    rescale=1./255,
    )

    train = train.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return train

In [7]:
def val_dg(path):

    test = ImageDataGenerator(
        rescale=1./255
    )

    test = test.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return test

In [8]:
train_datagen = train_dg(r"E:\archive\data\train")

Found 2942 images belonging to 3 classes.


In [9]:
test_datagen = val_dg(r"E:\archive\data\val")

Found 737 images belonging to 3 classes.


In [ ]:
print(train_datagen.class_indices)

In [ ]:
def conv(x, filters, kernel_size, strides):
    x = Conv2D( filters = filters, kernel_size = kernel_size, strides= strides, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def depv(x, filters, kernel_size, strides):
    x = DepthwiseConv2D(kernel_size=kernel_size, strides=strides,  padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters = filters, kernel_size = (1, 1), strides= 1, padding='same', activation="relu", use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

In [ ]:
def model(x):
    x = conv(x, 32, 3, 2)
    x = depv(x, 64, 3, 1)
    x = depv(x, 128, 3, 2)
    x = depv(x, 128, 3, 1)
    x = depv(x, 256, 3, 2)
    x = depv(x, 256, 3, 1)
    x = depv(x, 512, 3, 2)
    for i in range(5):
        x = depv(x, 512, 3, 1)
    x = depv(x, 1024, 3, 2)
    x = depv(x, 1024, 3, 2)
    # x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=1024, activation="relu")(x)
    x = Dense(units=3, activation="softmax", use_bias=False)(x)
    return x

In [ ]:
input = Input(shape=(224, 224, 3))
output = model(input)
cnn = Model(inputs=input, outputs=output)

In [ ]:
cnn.summary()

In [ ]:
cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)

In [12]:
cnn.fit(x=train_datagen, validation_data=test_datagen, epochs=30, batch_size=32, callbacks=[callback])

Epoch 1/30


92/92 [==============================] - 250s 3s/step - loss: 0.6431 - accuracy: 0.7175 - val_loss: 1.1008 - val_accuracy: 0.3148
Epoch 2/30
92/92 [==============================] - 205s 2s/step - loss: 0.4351 - accuracy: 0.8253 - val_loss: 1.0991 - val_accuracy: 0.3148
Epoch 3/30
92/92 [==============================] - 200s 2s/step - loss: 0.3455 - accuracy: 0.8712 - val_loss: 1.0996 - val_accuracy: 0.3148
Epoch 4/30
92/92 [==============================] - 230s 3s/step - loss: 0.2671 - accuracy: 0.9062 - val_loss: 1.1715 - val_accuracy: 0.3066
Epoch 5/30
92/92 [==============================] - 220s 2s/step - loss: 0.2445 - accuracy: 0.9092 - val_loss: 2.1846 - val_accuracy: 0.3066
Epoch 6/30
92/92 [==============================] - 234s 3s/step - loss: 0.2273 - accuracy: 0.9283 - val_loss: 2.2198 - val_accuracy: 0.3066
Epoch 7/30
92/92 [==============================] - 256s 3s/step - loss: 0.2053 - accuracy: 0.9290 - val_loss: 0.5183 - val_accuracy: 0.7693
Epoch 8/30


In [13]:
cnn.save("mobilenet.h5")

C:\Users\tharu\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model = tf.keras.models.load_model('mobilenet.h5', custom_objects=get_custom_objects())

TypeError: Error when deserializing class 'Conv2D' using config={'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'filters': 32, 'kernel_size': [3, 3], 'strides': [2, 2], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'relu', 'use_bias': False, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}.

Exception encountered: module 'keras.src.backend' has no attribute 'RandomGenerator'

In [13]:
test_image =image.load_img(r"E:\wheat disease\data\val\Healthy\Healthy057.jpg", target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image.shape)
result = model.predict(test_image)
print(result)

(1, 224, 224, 3)
1/1 [==============================] - 1s 528ms/step
[[0. 1. 0.]]


In [14]:
print(test_datagen.class_indices)

{'Brown_rust': 0, 'Healthy': 1, 'Yellow_rust': 2}
